In [2]:
import numpy as np
import xarray as xr
import pickle
import matplotlib.pyplot as plt
import sys
# import eddytools as et
import datetime as dt
from joblib import Parallel, delayed
from scipy import stats as st
import cmocean.cm as cmo

In [ ]:
datapath='/PATH/TO/DATA/'
figpath='/PATH/TO/OUTPUT/'

In [ ]:
#saved from fig6 notebook
with open(datapath+'so3_et_tracks_4d_types_processed.pickle','rb') as f:
        out_accy=pickle.load(f)

In [ ]:
# select the datasets: refer to extract func for order (fig_5)
#this can be improved
out_ac=[out_accy[0][1],out_accy[2][1],out_accy[4][1],out_accy[6][1],
          out_accy[1][1],out_accy[3][1],out_accy[5][1],out_accy[7][1]]
out_cy=[out_accy[0][0],out_accy[2][0],out_accy[4][0],out_accy[6][0],
          out_accy[1][0],out_accy[3][0],out_accy[5][0],out_accy[7][0]]

In [ ]:
#definitions for extracting data
def getmonths(data):    
    dates=[dt.datetime(year=int(str(track['time'][int(len(track['time'])/2)])[:4]),
                      month=int(str(track['time'][int(len(track['time'])/2)])[5:7]),
                      day=15).month for track in data]
    return np.asarray(dates)

def getsizes(data):    
    sizes=[np.mean(track['area']) for track in data]
    return np.asarray(sizes)

def getamps(data):    
    amps=[np.mean(track['amp']) for track in data]
    return np.asarray(amps)

In [ ]:
#extract data
outdatescy=[]
outdatesac=[]
outareascy=[]
outareasac=[]
outampscy = []
outampsac =[]
for i in np.arange(len(out_cy)):
    outdatescy.append(getmonths(out_cy[i]))
    outdatesac.append(getmonths(out_ac[i]))
    
    outareascy.append(getsizes(out_cy[i]))
    outareasac.append(getsizes(out_ac[i]))
    
    outampscy.append(getamps(out_cy[i]))
    outampsac.append(getamps(out_ac[i]))

In [ ]:
#func to get the p value of a t test between the 1951-1956 and 2091-2096 mean
#this should be combined with means and other stats
def significance(month,depth,ddata,tdata):
    # select time period (0=1950s or 1=2090s) and depth 0=0,1=25 etc
    ddata0=ddata[0:4]
    ddata1=ddata[4:]
    tdata0=tdata[0:4]
    tdata1=tdata[4:]
    
    ddata0=ddata0[depth]
    ddata1=ddata1[depth]
    tdata0=tdata0[depth]
    tdata1=tdata1[depth]

    data0monthly=np.asarray(ddata0)[np.asarray(tdata0)==month]
    data1monthly=np.asarray(ddata1)[np.asarray(tdata1)==month]
    
    return st.ttest_ind(data0monthly,data1monthly,equal_var=False)[1]
    

In [ ]:
#funcs for mean
def mean50(month,depth,ddata,tdata):
    # select time period (0=1950s ) and depth 0=0,1=25 etc
    ddata0=ddata[0:4]
    tdata0=tdata[0:4]
    
    ddata0=ddata0[depth]
    tdata0=tdata0[depth]

    data0monthly=np.asarray(ddata0)[np.asarray(tdata0)==month]
    
    return np.nanmean(data0monthly)

def mean90(month,depth,ddata,tdata):
    # select time period (0=1950s ) and depth 0=0,1=25 etc
    ddata1=ddata[4:]
    tdata1=tdata[4:]
    
    ddata1=ddata1[depth]
    tdata1=tdata1[depth]

    data1monthly=np.asarray(ddata1)[np.asarray(tdata1)==month]
    
    return np.nanmean(data1monthly)

In [ ]:
#construct arrays of significance and mean for each variable: 12 months by 4 depths
areasigac=[]
areasigcy=[]
amp_sigac=[]
amp_sigcy=[]

areamean90ac=[]
areamean90cy=[]
amp_mean90ac=[]
amp_mean90cy=[]

areamean50ac=[]
areamean50cy=[]
amp_mean50ac=[]
amp_mean50cy=[]

for dind in [3,2,1,0]:
    tempsig_arac=[]
    tempsig_arcy=[]
    tempsig_amac=[]
    tempsig_amcy=[]
    
    tempmean50_arac=[]
    tempmean50_arcy=[]
    tempmean50_amac=[]
    tempmean50_amcy=[]

    tempmean90_arac=[]
    tempmean90_arcy=[]
    tempmean90_amac=[]
    tempmean90_amcy=[]
    for mind in np.arange(1,13,1):
        tempsig_arac.append(significance(mind,dind,outareasac,outdatesac))
        tempsig_arcy.append(significance(mind,dind,outareascy,outdatescy))
        tempsig_amac.append(significance(mind,dind,outampsac,outdatesac))
        tempsig_amcy.append(significance(mind,dind,outampscy,outdatescy))
        
        tempmean50_arac.append(mean50(mind,dind,outareasac,outdatesac))
        tempmean50_arcy.append(mean50(mind,dind,outareascy,outdatescy))
        tempmean50_amac.append(mean50(mind,dind,outampsac,outdatesac))
        tempmean50_amcy.append(mean50(mind,dind,outampscy,outdatescy))
        
        tempmean90_arac.append(mean90(mind,dind,outareasac,outdatesac))
        tempmean90_arcy.append(mean90(mind,dind,outareascy,outdatescy))
        tempmean90_amac.append(mean90(mind,dind,outampsac,outdatesac))
        tempmean90_amcy.append(mean90(mind,dind,outampscy,outdatescy))
    
    areasigac.append(tempsig_arac)
    areasigcy.append(tempsig_arcy)
    amp_sigac.append(tempsig_amac)
    amp_sigcy.append(tempsig_amcy)
    
    areamean50ac.append(tempmean50_arac)
    areamean50cy.append(tempmean50_arcy)
    amp_mean50ac.append(tempmean50_amac)
    amp_mean50cy.append(tempmean50_amcy)
    
    areamean90ac.append(tempmean90_arac)
    areamean90cy.append(tempmean90_arcy)
    amp_mean90ac.append(tempmean90_amac)
    amp_mean90cy.append(tempmean90_amcy)

In [ ]:
areasigac=np.asarray(areasigac)
areasigcy=np.asarray(areasigcy)
amp_sigac=np.asarray(amp_sigac)
amp_sigcy=np.asarray(amp_sigcy)

areamean50ac=np.asarray(areamean50ac)
areamean50cy=np.asarray(areamean50cy)
amp_mean50ac=np.asarray(amp_mean50ac)
amp_mean50cy=np.asarray(amp_mean50cy)

areamean90ac=np.asarray(areamean90ac)
areamean90cy=np.asarray(areamean90cy)
amp_mean90ac=np.asarray(amp_mean90ac)
amp_mean90cy=np.asarray(amp_mean90cy)

In [ ]:
# plot

In [ ]:
#significance masks
sigs=[areasigcy,areasigac,amp_sigcy,amp_sigac]
sigs=[np.ma.masked_less(i,0.05)for i in sigs]

In [ ]:
monstrings=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
depthstrings=['190-200m','95-100m','25-30m','0-5m']
labels=['a','b','c','d','e','f','g']

In [ ]:
# colormaps - dark parts removed
rm = cmo.tools.crop_by_percent(cmo.amp, 10, which='max', N=None)
rbm = cmo.tools.crop_by_percent(cmo.balance, 10, which='both', N=None)

In [ ]:
plt.rc('xtick',labelsize=14)
plt.rc('ytick',labelsize=14)
plt.rc('axes',labelsize=14)

In [ ]:
#figure 6
fig,ax=plt.subplots(4,3,figsize=(16,10),dpi=300,sharex=True, sharey=True,constrained_layout=True)#height_ratios=[1,1,1,1.15])#constrained_layout=True)

titlesize=20
ax[0,0].set_title('1951-1956',fontsize=titlesize)
ax[0,1].set_title('2091-2096',fontsize=titlesize)
ax[0,2].set_title('Change',fontsize=titlesize)

#column 1
#0,0
d_00=ax[0,0].pcolormesh(monstrings,depthstrings,areamean50cy,cmap=rm,vmin=500,vmax=900)
plt.colorbar(d_00,ax=ax[0,0],orientation='horizontal',extend='both',label='Cyclone area ($km^2$)')

#0,1
d_01=ax[1,0].pcolormesh(monstrings,depthstrings,areamean50ac,cmap=rm,vmin=500,vmax=900)
plt.colorbar(d_01,ax=ax[1,0],orientation='horizontal',extend='both',label='Anticyclone area ($km^2$)')

#1,0
d_10=ax[2,0].pcolormesh(monstrings,depthstrings,amp_mean50cy*10e5,cmap=rm,vmin=2.,vmax=4.5)
plt.colorbar(d_10,ax=ax[2,0],orientation='horizontal',extend='both',label='Cyclone vorticity amplitude ($10^5 s^{-1}$)',ticks=[2.5,3.5,4.5],format=ticker.FixedFormatter(['$2.5$','$3.5$','$4.5$']))

#1,1
d_11=ax[3,0].pcolormesh(monstrings,depthstrings,amp_mean50ac*10e5,cmap=rm,vmin=2.,vmax=4.5)
plt.colorbar(d_11,ax=ax[3,0],orientation='horizontal',extend='both',label='Anticyclone vorticity amplitude ($10^5 s^{-1}$)',ticks=[2.5,3.5,4.5],format=ticker.FixedFormatter(['$2.5$','$3.5$','$4.5$']))

#column 2
#0,0
d_00=ax[0,1].pcolormesh(monstrings,depthstrings,areamean90cy,cmap=rm,vmin=500,vmax=900)
plt.colorbar(d_00,ax=ax[0,1],orientation='horizontal',extend='both',label='Cyclone area ($km^2$)')

#0,1
d_01=ax[1,1].pcolormesh(monstrings,depthstrings,areamean90ac,cmap=rm,vmin=500,vmax=900)
plt.colorbar(d_01,ax=ax[1,1],orientation='horizontal',extend='both',label='Anticyclone area ($km^2$)')

#1,0
d_10=ax[2,1].pcolormesh(monstrings,depthstrings,amp_mean90cy*10e5,cmap=rm,vmin=2.,vmax=4.5)
plt.colorbar(d_10,ax=ax[2,1],orientation='horizontal',extend='both',label='Cyclone vorticity amplitude ($10^5 s^{-1}$)',ticks=[2.5,3.5,4.5],format=ticker.FixedFormatter(['$2.5$','$3.5$','$4.5$']))

#1,1
d_11=ax[3,1].pcolormesh(monstrings,depthstrings,amp_mean90ac*10e5,cmap=rm,vmin=2.,vmax=4.5)
plt.colorbar(d_11,ax=ax[3,1],orientation='horizontal',extend='both',label='Anticyclone vorticity amplitude ($10^5 s^{-1}$)',ticks=[2.5,3.5,4.5],format=ticker.FixedFormatter(['$2.5$','$3.5$','$4.5$']))

#column 3
#0,0
d_00=ax[0,2].pcolormesh(monstrings,depthstrings,areamean90cy-areamean50cy,cmap=rbm,vmin=-250,vmax=250)
plt.colorbar(d_00,ax=ax[0,2],orientation='horizontal',extend='both',label='Δ Cyclone area ($km^2$)')

#0,1
d_01=ax[1,2].pcolormesh(monstrings,depthstrings,areamean90ac-areamean50ac,cmap=rbm,vmin=-250,vmax=250)
plt.colorbar(d_01,ax=ax[1,2],orientation='horizontal',extend='both',label='Δ Anticyclone area ($km^2$)')

#1,0
d_10=ax[2,2].pcolormesh(monstrings,depthstrings,amp_mean90cy*10e5-amp_mean50cy*10e5,cmap=rbm,vmin=-1.2,vmax=1.2)
plt.colorbar(d_10,ax=ax[2,2],orientation='horizontal',extend='both',label='Δ Cyclone vorticity amplitude ($10^5 s^{-1}$)',ticks=[-1,0,1],format=ticker.FixedFormatter(['$-1$','0','$1$']))

#1,1
d_11=ax[3,2].pcolormesh(monstrings,depthstrings,amp_mean90ac*10e5-amp_mean50ac*10e5,cmap=rbm,vmin=-1.2,vmax=1.2)
plt.colorbar(d_11,ax=ax[3,2],orientation='horizontal',extend='both',label='Δ Anticyclone vorticity amplitude ($10^5 s^{-1}$)',ticks=[-1,0,1],format=ticker.FixedFormatter(['$-1$','0','$1$']))

#hatching
for i in np.arange(4):
    ax[i,2].pcolor(monstrings,depthstrings,sigs[i],hatch='/',alpha=0)

#labels
for num,axis in enumerate(ax[:,0]):
    axis.annotate(labels[num], xy=(-0.2, 1.15),xycoords='axes fraction',horizontalalignment='left', 
                     verticalalignment='top',fontsize=22,weight='bold')
    
plt.savefig(figpath+'fig_6.png',bbox_inches='tight')